ยินดีด้วยครับที่คุณก้าวข้ามจากข้อมูลแบบ **Vector** (จุด, เส้น, พื้นที่) เข้าสู่โลกของ **Raster** ซึ่งเป็นหัวใจสำคัญของงานวิเคราะห์ภาพถ่ายดาวเทียมและข้อมูลเชิงพื้นที่แบบตารางครับ

---

## **17.1 บทนำสู่ Rasterio**

หาก GeoPandas คือราชาแห่งข้อมูล Vector... **Rasterio** ก็คือราชาแห่งข้อมูล **Raster** ในโลกของ Python ครับ มันถูกสร้างขึ้นมาเพื่อให้เราจัดการกับไฟล์ภาพถ่ายทางภูมิศาสตร์ (เช่น `.tif`) ได้ง่ายและรวดเร็ว โดยอาศัยความสามารถของโปรแกรมระดับตำนานอย่าง **GDAL** อยู่เบื้องหลัง

### **17.1.1 ข้อมูล Raster คืออะไร?**

ให้คุณนึกถึง **"รูปภาพ"** ครับ ข้อมูลแบบ Raster คือตารางที่ประกอบด้วยจุดเล็กๆ ที่เรียกว่า **Pixel (พิกเซล)** แต่ละพิกเซลจะเก็บ "ค่าตัวเลข" ที่แทนข้อมูลในโลกจริงไว้

**ตัวอย่างสิ่งที่เรามักเก็บในรูปแบบ Raster:**

* **Elevation (DEM):** แต่ละพิกเซลบอกความสูงของพื้นที่ (เช่น พิกเซลนี้สูง 500 เมตรจากระดับน้ำทะเล)
* **Satellite Imagery:** เก็บค่าการสะท้อนแสง (เช่น แบนด์สีแดง, เขียว, น้ำเงิน)
* **Climate Data:** แต่ละพิกเซลบอกอุณหภูมิหรือปริมาณน้ำฝน

**4 ลักษณะเด่นของ Raster:**

1. **Spatial Resolution (ความละเอียด):** พิกเซลหนึ่งกว้างกี่เมตรในโลกจริง? (เช่น 10m x 10m)
2. **Extent (ขอบเขต):** ภาพนี้ครอบคลุมพื้นที่จากไหนถึงไหน?
3. **CRS (ระบบพิกัด):** ตารางพิกเซลนี้วางอยู่ตรงไหนของแผนที่โลก?
4. **Bands (จำนวนชั้น):** ภาพมีกี่ชั้น? (เช่น ภาพถ่ายดาวเทียมอาจมี 10 ชั้นที่เก็บค่าแสงคนละช่วงคลื่น)

---

### **17.1.2 ทำไมต้องใช้ Rasterio?**

การใช้ Python เปิดภาพ `.jpg` ธรรมดานั้นทำได้ง่าย แต่ภาพดาวเทียมมักจะมีขนาด **ใหญ่มาก (หลาย GB)** และมี **พิกัด** ติดมาด้วย

* **ประหยัดแรม:** Rasterio สามารถ "ค่อยๆ อ่าน" ข้อมูลทีละส่วนได้ ไม่ต้องโหลดทั้งภาพเข้าแรม
* **ไม่ทำพิกัดหาย:** มันจะรักษาค่าละติจูด/ลองจิจูดของภาพไว้เสมอ
* **ทำงานร่วมกับ NumPy ได้ทันที:** ข้อมูลในพิกเซลจะถูกแปลงเป็น `numpy array` ทำให้เราคำนวณทางคณิตศาสตร์ได้ง่ายมาก

---

## **17.2 สิ่งที่คุณจะได้เรียนในบทนี้**

เมื่อจบเนื้อหาส่วนนี้ คุณจะสามารถ:

1. **อ่าน-เขียนไฟล์ Raster:** โดยที่พิกัดยังอยู่ครบถ้วน
2. **จัดการ Metadata:** ดูค่าทางสถิติและข้อมูลทางภูมิศาสตร์ของภาพ
3. **สร้างภาพ (Visualization):** ระบายสีภาพถ่ายดาวเทียมหรือข้อมูลความสูงให้สวยงาม
4. **Band Math:** คำนวณข้ามแบนด์ (เช่น การหาดัชนีความเขียวขจีของพืชพรรณ หรือ NDVI)

---



ก่อนจะเข้าสู่การประมวลผลภาพถ่ายดาวเทียมหรือข้อมูลความสูง ขั้นตอนแรกคือการเตรียมเครื่องมือให้พร้อมครับ ในโลกของ Python การจัดการ Raster ต้องอาศัยการทำงานร่วมกันของหลายห้องสมุด (Libraries) 5555 เพื่อให้ครอบคลุมทั้งการอ่านไฟล์ การคำนวณ และการแสดงผล

---

## **17.3 การติดตั้งและการเตรียมสภาพแวดล้อม**

### **การติดตั้ง**

หากคุณใช้งานบน Jupyter Notebook หรือ Google Colab ให้รันคำสั่งนี้เพื่อติดตั้ง Rasterio และเครื่องมือเสริมที่จำเป็น:

```python
# ติดตั้ง rasterio และ pygis (ซึ่งรวมเครื่องมือด้าน GIS ที่เป็นประโยชน์ไว้มากมาย)
%pip install rasterio pygis

```

---

### **การ Import และหน้าที่ของแต่ละ Library**

หัวใจสำคัญของการวิเคราะห์ Raster ไม่ได้มีแค่ Rasterio เพียงอย่างเดียว แต่ต้องใช้ "ทีมเวิร์ก" ดังนี้ครับ:

```python
import rasterio
import rasterio.plot
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

```

* **`rasterio`**: ทำหน้าที่เป็น "ประตู" ในการเปิด-ปิด และอ่านค่าพิกัด/ข้อมูลภายในไฟล์ Raster (เช่น `.tif`, `.img`)
* **`rasterio.plot`**: ตัวช่วยพิเศษที่ทำให้การสั่งวาดภาพจากข้อมูลพิกัด Raster ทำได้ง่ายขึ้นมาก โดยไม่ต้องตั้งค่าเองเยอะ
* **`geopandas`**: แม้เราจะเน้น Raster แต่บ่อยครั้งเราต้องใช้ข้อมูล Vector (เช่น ขอบเขตจังหวัด) มาซ้อนทับหรือตัดภาพ (Clipping)
* **`numpy`**: **สำคัญมาก!** เพราะ Rasterio จะอ่านข้อมูลพิกเซลออกมาเป็น `numpy array` ทำให้เราสามารถคำนวณคณิตศาสตร์ได้รวดเร็วเหมือนการคำนวณตารางตัวเลข
* **`matplotlib`**: ใช้สำหรับสร้างกราฟ ปรับแต่งสีแผนที่ และจัดวาง Layout ของรูปภาพให้น่าสนใจ

---

### **ทำไมต้องทำงานร่วมกับ NumPy?**

เมื่อคุณเปิดไฟล์ภาพด้วย Rasterio ข้อมูลในแต่ละพิกเซลจะถูกมองเป็น "ตัวเลข" ในอาร์เรย์ของ NumPy ทันที

การแปลงภาพให้เป็นตัวเลขแบบนี้มีประโยชน์มหาศาลครับ เช่น:

* **Band Math**: การเอาพิกเซลจากแบนด์อินฟราเรดมาลบกับแบนด์สีแดง (เพื่อดูพื้นที่ป่าไม้)
* **Filtering**: การลดจุดรบกวนในภาพ
* **Reclassification**: การเปลี่ยนค่าพิกเซลจาก "ความสูง" ให้กลายเป็น "ระดับความเสี่ยงน้ำท่วม" (เช่น 1, 2, 3)

---



In [ ]:
# pip install rasterio geopandas pygis

In [1]:
import rasterio
import rasterio.plot
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

ImportError: DLL load failed while importing lib: The specified procedure could not be found.

In [3]:
import os; print(os.environ['PATH'])

c:\Users\thammarat\miniconda3\envs\geo_new;C:\Users\thammarat\miniconda3\envs\geo_new;C:\Users\thammarat\miniconda3\envs\geo_new\Library\mingw-w64\bin;C:\Users\thammarat\miniconda3\envs\geo_new\Library\usr\bin;C:\Users\thammarat\miniconda3\envs\geo_new\Library\bin;C:\Users\thammarat\miniconda3\envs\geo_new\Scripts;C:\Users\thammarat\miniconda3\envs\geo_new\bin;C:\Users\thammarat\miniconda3\condabin;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0;C:\WINDOWS\System32\OpenSSH;C:\Users\thammarat\miniconda3;C:\Users\thammarat\miniconda3\Scripts;C:\Users\thammarat\miniconda3\Library\bin;C:\Program Files\Git\cmd;C:\Program Files\nodejs;C:\Program Files\Docker\Docker\resources\bin;C:\Users\thammarat\AppData\Local\Programs\Python\Python310\Scripts;C:\Users\thammarat\AppData\Local\Programs\Python\Python310;C:\Users\thammarat\.local\bin;C:\Users\thammarat\AppData\Local\Programs\Python\Python314\Scripts;C:\Users\thammarat\AppData\Local\Programs\Py

In [4]:
import os
print("Env path:", os.environ.get('CONDA_PREFIX'))
dll_dir = os.path.join(os.environ.get('CONDA_PREFIX', ''), 'Library', 'bin')
print("DLL dir exists?", os.path.exists(dll_dir))


Env path: C:\Users\thammarat\miniconda3\envs\geo_new
DLL dir exists? True


In [5]:
!conda list geopandas

# packages in environment at C:\Users\thammarat\miniconda3\envs\geo_new:
#
# Name                     Version          Build            Channel
geopandas                  1.1.1            pyhd8ed1ab_1     conda-forge
geopandas-base             1.1.1            pyha770c72_1     conda-forge


In [6]:
import os
print("CONDA_PREFIX:", os.environ.get('CONDA_PREFIX'))
print("PATH has Library/bin?", 'Library\\bin' in os.environ['PATH'])


CONDA_PREFIX: C:\Users\thammarat\miniconda3\envs\geo_new
PATH has Library/bin? True


In [7]:
gpd.read_file("example.shp")

NameError: name 'gpd' is not defined

In [8]:
print(os.environ['PATH']) 

c:\Users\thammarat\miniconda3\envs\geo_new;C:\Users\thammarat\miniconda3\envs\geo_new;C:\Users\thammarat\miniconda3\envs\geo_new\Library\mingw-w64\bin;C:\Users\thammarat\miniconda3\envs\geo_new\Library\usr\bin;C:\Users\thammarat\miniconda3\envs\geo_new\Library\bin;C:\Users\thammarat\miniconda3\envs\geo_new\Scripts;C:\Users\thammarat\miniconda3\envs\geo_new\bin;C:\Users\thammarat\miniconda3\condabin;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0;C:\WINDOWS\System32\OpenSSH;C:\Users\thammarat\miniconda3;C:\Users\thammarat\miniconda3\Scripts;C:\Users\thammarat\miniconda3\Library\bin;C:\Program Files\Git\cmd;C:\Program Files\nodejs;C:\Program Files\Docker\Docker\resources\bin;C:\Users\thammarat\AppData\Local\Programs\Python\Python310\Scripts;C:\Users\thammarat\AppData\Local\Programs\Python\Python310;C:\Users\thammarat\.local\bin;C:\Users\thammarat\AppData\Local\Programs\Python\Python314\Scripts;C:\Users\thammarat\AppData\Local\Programs\Py

In [9]:
import os
# Path env ของคุณ
env_root = r"C:\Users\thammarat\miniconda3\envs\geo_new"
dll_path = os.path.join(env_root, "Library", "bin")
os.add_dll_directory(dll_path)
print(f"✅ DLL path added: {dll_path}")
print(f"✅ Exists: {os.path.exists(dll_path)}")


✅ DLL path added: C:\Users\thammarat\miniconda3\envs\geo_new\Library\bin
✅ Exists: True


In [10]:
import geopandas as gpd
print(f"✅ GeoPandas version: {gpd.__version__}")
gpd.plotting.plot_dataframe  # ทดสอบ plot


ImportError: DLL load failed while importing lib: The specified procedure could not be found.

In [11]:
import os
env_root = r"C:\Users\thammarat\miniconda3\envs\geo_new"
os.add_dll_directory(os.path.join(env_root, "Library", "bin"))


<AddedDllDirectory('C:\\Users\\thammarat\\miniconda3\\envs\\geo_new\\Library\\bin')>

In [1]:
import geopandas as gpd
print(f"GeoPandas: {gpd.__version__}")


GeoPandas: 1.1.1
